In [ ]:
pos_sentences = [
    "Sam sneezed the napkin off the table.",
    "Joe squeezed the rubber ball inside the jar.",
    "She put the phone on the desk.",
    "Pat pushed the piano into the room.",
    "Pat ordered him into the room.",
    "Pat allowed Chris into the room.",
    "Pat blocked Chris out of the room.",
    "Pat assisted Chris into the room.",
    "Pat threw the metal off the table.",
    "Joe moved it onto the table.",
    "He threw it towards the door.",
    "He put it near the table.",
    "Joe kicked the bottle into the yard.",
    "They laughed the poor guy out of the room.",
    "Sue let the water out of the bathtub.",
    "Sam helped him into the car.",
    "They sprayed the paint onto the wall.",
    "Joe kicked the dog into the bathroom.",
    "Frank squeezed the ball through the crack.",
    "Sam sawed/tore/hacked/ripped a piece off the block.",
    "Sam washed/rinsed/cleaned the soap out of her eyes.",
    "Sam mixed/stirred the paint thinner into the paint.",
    "The audience laughed the poor guy off of the stage.",
    "In the last star trek episode, there was a woman who could think people into a different galaxy.",
    "Fred stuffed the papers in the envelope.",
    "Sam pushed him within arm's length of the grenade.",
    "Sam shoved him outside the room.",
    "Sam urged Bill outside of the house.",
    "Sam shoved it into the carton.",
    "Sam invited him out to her cabin.",
    "Sam beckoned him into the room.",
    "Sam urged him into the room.",
    "Sam sent him to the market.",
    "Sam let Bill into the room.",
    "Sam blocked Joe out of the house.",
    "Sam locked him into the bathroom.",
    "He kept her at arm's length.",
    "Sam barricaded him out of the room.",
    "Sam assisted her out of the room.",
    "Sam guided him through the terrain.",
    "Sam showed him into the living room.",
    "Sam walked him to the car.",
    "Sam accompanied Bob into the room.",
    "Ann chased the squirrel out of her house.",
    "Sam coaxed him into the room.",
    "He hit the ball over the fence.",
    "Please chop the kindling into the bin provided for it.",
    "Sam asked him into the room.",
    "Joe pushed the piano up the stairs.",
    "The wind blew the ship off course.",
    "The rain swept the ring into the gutter.",
    "Sam frightened Bob out of the house.",
    "Sam lured him into the room.",
    "Sam invited him onto the deck.",
    "Sam permitted him out of the house.",
    "Sam asked Joe into the room.",
    "Sam ordered Bob into the jail cell.",
    "The company flew her to Chicago for an interview.",
    "He hit the ball across the field.",
    "The butcher sliced the salami onto the wax paper.",
    "Joey clumped his potatoes into the middle of his plate.",
    "Joey grated the cheese onto a serving plate.",
    "Sam shredded the papers into the garbage pail.",
    "Sam carefully broke the eggs into the bowl.",
    "He nudged the gold ball into the hole.",
    "He shoved the cart down the incline.",
    "Sam frightened Bob away from the door.",
    "They laughed the poor guy into his car.",
    "Sam, stop frightening Bobby under the bed."
]


In [ ]:
neg_sentences = [
    'Joe careened into the room with the help of cane.',
    'The boat burned into the cave.',
    'The bird chirped out of the door.',
    'The dog barked into the room.',
    'The rooster crowed out of the barn.',
    'The man laughed out of room.',
    'Joe struck the ball across the field.',
    'Sam encouraged him into the room.',
    'He struck the ball over the fence.',
    'Sam begged him into the room.',
    'The hammer broke the vase onto the floor.',
    'The hammer broke the vase into pieces.',
    'His cane helped him into the car.',
    'Sam convinced/persuaded/encouraged/instructed him into the room.',
    'Sam pleaded Joe into the room.',
    'Farmer Joe grew those vines onto his roof.',
    'The invalid owner ran his favorite horse onto the field.',
    'The car struck a brick wall into pieces.',
    'He filled water into the tub.',
    'He covered the blanket over Mary.',
    'He nudged the ball down the incline.',
    'They laughed the poor guy into his car.',
    'Sam frightened Bob under the bed.']


In [ ]:
adpositions = ['across', 'at', 'down', 'from', 'in', 'inside', 'into', 'near', 'off', 'off of', 'on', 'onto', 'out of', 'out to', 'outside', 'outside of', 'over', 'through', 'to', 'towards', 'under', 'up', 'within']

def check_dobj(i, dependencies, verb):
    if i >= len(dependencies):
        return None
    text, lemma, pos, dep, head, head_i = dependencies[i]

    if pos in ['VERB', 'AUX']:
        return None
    
    if dep == 'ROOT':
        return None

    if dep == 'dobj' and head == verb:
        return i

    return check_dobj(head_i, dependencies, verb) 


def check_adp(start_i, dependencies, adpositions): #note that this does not include dependencies because spacy can't be trusted on those
    if start_i + 1 < len(dependencies):
        longer_adp = dependencies[start_i][0] + " " + dependencies[start_i + 1][0]
        if longer_adp in adpositions:
            return ([start_i, start_i + 1], longer_adp)
    
    if start_i < len(dependencies) and dependencies[start_i][0] in adpositions:
        return ([start_i], dependencies[start_i][1])

    return (None, None)

def check_pobj(dependencies, adp_index):
    pobj_token_list = [(lemma, pobj_i) for pobj_i, (text, lemma, pos, dep, head, head_i) in enumerate(dependencies) if head_i in adp_index and dep == 'pobj' and pos not in ['NUM', 'PUNCT']]
    if len(pobj_token_list) > 1:
        raise Exception
    if len(pobj_token_list) == 1:
        return pobj_token_list[0]
    return (None, None)

# Why     why     SCONJ   advmod  is      1
def is_cm(dependencies, nlp, sentence):
    for i, (token_text, lemma, pos, dep, head, head_i) in enumerate(dependencies):
        if pos == 'VERB':
            # Look for a direct object (dobj) immediately following the verb
            dobj_i = check_dobj(i + 1, dependencies, token_text)
            if dobj_i:
                dobj_lemma = dependencies[dobj_i][1]
                # Check if the following token is an adposition (ADP) from the list
                (adp_index, adp_lemma) = check_adp(dobj_i + 1, dependencies, adpositions)
                if adp_index:
                    # do another spacy dependency parse
                    doc = nlp(sentence)
                    dependencies = [(token.text, token.lemma_, token.pos_, token.dep_, token.head.text, token.head.i) for token in doc]
                    pobj_lemma, pobj_i = check_pobj(dependencies, adp_index)
                    if pobj_lemma:
                        result_dict = {'verb': lemma, 'verb_i' : i, 'direct_object': dobj_lemma, 'direct_object_i': dobj_i, 'preposition': adp_lemma, 'preposition_i': adp_index, 'prepositional_object': pobj_lemma, 'prepositional_object_i': pobj_i}
                        return result_dict
                    # Look for a prepositional object (pobj) child of the adp



    return None

In [ ]:
import spacy
import json
nlp = spacy.load("en_core_web_sm")

def expand_slashes(sentences):
    new_sentences = []
    for sentence in sentences:
        words = sentence.split()
        for i, word in enumerate(words):
            if "/" in word:
                options = word.split("/")
                for option in options:
                    new_words = words[:i] + [option.strip()] + words[i+1:]
                    new_sentence = " ".join(new_words)
                    new_sentences.append(new_sentence)
                break
        else:
            new_sentences.append(sentence)
    return new_sentences


# Function to perform dependency parsing
def dependency_parsing(sentences):
    parsed_sentences = []
    for sentence in sentences:
        doc = nlp(sentence)
        dependencies = [(token.text, token.lemma_, token.pos_, token.dep_, token.head.text, token.head.i) for token in doc]
        parsed_sentences.append({"sentence": sentence, "dependencies": dependencies})
    return parsed_sentences



In [ ]:
processed_sentences_positive = dependency_parsing(expand_slashes(pos_sentences))
processed_sentences_negative = dependency_parsing(expand_slashes(neg_sentences))

checked_sentences_positive = [(sentence, is_cm(sentence["dependencies"], nlp, sentence["sentence"])) for sentence in processed_sentences_positive]
checked_sentences_negative = [(sentence, is_cm(sentence["dependencies"], nlp, sentence["sentence"])) for sentence in processed_sentences_negative]

my_sentences = ["Just do n't limit it to the last 10 games .", "None of my family noticed anything out of the ordinary .", "Surely this wastes beer through unnecessary spillages .","Sturridge has missed 74 games through injury in 3 seasons ? !","Refuse to talk politics at work ."]
my_sentences_parsed = dependency_parsing(my_sentences)
my_sentences_checked = [(sentence, is_cm(sentence["dependencies"], nlp, sentence["sentence"])) for sentence in my_sentences_parsed]
for sentence, result in my_sentences_checked:
    if result:
        print(sentence["sentence"], end = "\t")
        print(f"{result['verb']}\t{result['direct_object']}\t{result['preposition']}\t{result['prepositional_object']}\t{result['verb_i']}\t{result['direct_object_i']}\t{result['preposition_i']}\t{result['prepositional_object_i']},1")
for sentence, result in checked_sentences_positive:
    if result:
        print(sentence["sentence"], end = "\t")
        print(f"{result['verb']}\t{result['direct_object']}\t{result['preposition']}\t{result['prepositional_object']}\t{result['verb_i']}\t{result['direct_object_i']}\t{result['preposition_i']}\t{result['prepositional_object_i']},1")
       

In [ ]:
for sentence, result in checked_sentences_negative:
    if result:
        print(sentence["sentence"], end = ",")
        print(f"{result['verb']},{result['direct_object']},{result['preposition']},{result['prepositional_object']},{result['verb_i']},{result['direct_object_i']},{result['preposition_i']},{result['prepositional_object_i']},0")
        